<a href="https://colab.research.google.com/github/Om-Shree-0709/Pytorch/blob/main/PCA_%26_its_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PCA : principal component analysis [Pre-Processing Techniques] \
D:{x,y} : given dataset (supervised dataset)
X belongs to R raised to power d [features] : dimensionality->x ke features

D conveted to D' : {x,y} X belongs to R raised to power d' \
( where d'< d always )

TO REDUCE THE DIMENSIONALITY OF FEATURES \
because : faster computation and training

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
